Muslimov Arthur, Last Checkpoint: 17/12/2021  </br>

Один из важнейших плюсов NumPy - это его быстрота и эффективность при поэлементных  <br/>
операциях. Эта возможность частично перекочевала и в Pandas. При этом Pandas        <br/>
добавляет несколько своих полезных трюков - а именно то, что результат операций     <br/>
не забывает, что он Pandas, а значит метки строк и столбцов остаются при нём.       <br/>
Ещё важно то, что при бинарных операциях, например, сложении, Pandas будет          <br/>
автоматически ***выравнивать индексы***, т.е. ты защищён от возможных ошибок при        <br/>
передаче и объединении данных, взятых с различных источников.

In [1]:
import pandas as pd
import numpy as np

%xmode Minimal
%autosave 0

Exception reporting mode: Minimal


Autosave disabled


## Универсальные функции: сохранение индекса

Т.к. библиотека Pandas предназначена для работы с библиотекой    <br/>
NumPy, абсолютно все универсальные функции NumPy будут работать  <br/>
с объектами Series и DataFrame библиотеки Pandas.

In [8]:
rng = np.random.Generator(np.random.MT19937(42))  # np.random.Generator() - стандартный способ
                                                  #   создать генератор на текущий момент (2021)
                                                  # np.random.MT19937(42) - выдаст генератор
                                                  #   Mersenne Twister, что используется в stdlib
                                                  #   Python'а. Seed я выставил 42
df = pd.DataFrame(rng.integers(0, 10, (2, 4)),    # или rng.randint(low=0, high=10, size=(2,4))
                  columns=['A', 'B', 'C', 'D'])   #   ****больше это не работает
df

,A,B,C,D
0,5,9,6,4
1,0,7,8,3


Результатом применением универсальных функций будет другой Pandas-объект.

In [9]:
np.sin(df * np.pi / 4)  # имена индексов и столбцов сохраняются

,A,B,C,D
0,-0.707107,0.707107,-1.000000e+00,1.224647e-16
1,0.000000,-0.707107,-2.449294e-16,7.071068e-01


## Универсальные функции: выравнивание индексов

При бинарных операциях над объектами DataFrame или Series библиотека Pandas  <br/>
будет выравнивать индексы в процессе. Это очень удобно при неполных данных.

### Выравнивание индексов в объектах Series

Допустим, ты откопал различные данные о штатах США.

In [10]:
area = pd.Series({'Alaska': 1723337, 'Texas': 695662,
                  'California': 423967}, name='area')  # да, имена тоже можно давать
population = pd.Series({'California': 38332521, 'Texas': 26448193,
                        'New York': 19651127}, name='population')

Вот незадача ... данные о штатах не симметричны. Посмотрим что  <br/>
выйдет, если попробовать подсчитать плотность населения.

In [11]:
density = population / area  # здесь они тоже видут себя как массивы
density

Alaska              NaN
California    90.413926
New York            NaN
Texas         38.018740
dtype: float64

К счастью нас никто не обругал. Pandas просто поставил `NaN` там, где нехватало  <br/>
данных. Мы даже можем определить штаты, о которых нам не хватает данных.

In [13]:
population.index ^ area.index  # штаты, о которых нам следует добыть побольше информации

<ipython-input-13-d685dbcde57c>:1: FutureWarning: Index.__xor__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__xor__.  Use index.symmetric_difference(other) instead
  population.index ^ area.index  # штаты, о которых нам следует добыть побольше информации


Index(['Alaska', 'New York'], dtype='object')

Подобным образом Pandas всегда будет ставить `NaN` там, где чего-то  <br/>
не хватает. Если ты не хочешь, чтобы в выводе красовались `NaN`,     <br/>
то можешь задать свою замену для отсутствующих данных.

In [14]:
population.divide(area, fill_value=10000)  # Раз у нас нет населения Аляски - пусть оно будет 10000.
    # делается это атрибутом fill_value    # Площади Нью-Йорка тоже нет. Пусть тоже будет 10000.
    # в методах объектов Pandas

Alaska           0.005803
California      90.413926
New York      1965.112700
Texas           38.018740
dtype: float64

### Выравнивание индексов  в объектах DataFrame

При операциях над объектами DataFrame происходит то же    <br/>
самое выравнивание как для столбцов, так и для индексов.

In [26]:
data_A = pd.DataFrame(rng.integers(0, 10, (2, 2)),
                      columns=list('AB'))  # да, так тоже можно
data_A

,A,B
0,6,7
1,9,2


In [27]:
data_B = pd.DataFrame(rng.integers(0, 10, (3, 3)), 
                      columns=list('BAC'))
data_B

,B,A,C
0,0,7,3
1,9,2,2
2,3,7,5


In [28]:
data_A + data_B  # или data_A.add(data_B)

,A,B,C
0,13.0,7.0,NaN
1,11.0,11.0,NaN
2,NaN,NaN,NaN


Как ты видишь, индексы выровнялись правильно, хотя и расположение     <br/>
у них было разное. В полученном результате они даже отсортированы.    <br/>
От `NaN` тоже можно избавиться, выставив свой заполнитель в методах.

In [29]:
fill = data_A.stack().mean()  # получаем среднее от элементов объекта data_A, предварительно
                         # выстроив значения data_A в один ряд. Можно было просто data_A.values.mean()
print('fill:', fill)
data_A.add(data_B, fill_value=fill)  # в качестве заполнителя используем среднее от data_A

fill: 6.0


,A,B,C
0,13.0,7.0,9.0
1,11.0,11.0,8.0
2,13.0,9.0,11.0


А вот и таблица операторов и эквивалентных методов объектов Pandas.  <br/>
Хм... Где это я уже видел... Да это же почти NumPy-методы!

**Оператор языка Python** | **Метод (-ы) библиотеки Pandas**
-----:|:-----
`+`   | `add()`
`-`   | `sub()`, `subtract()`
`*`   | `mul()`, `multiply()`
`\`   | `truediv()`, `div()`, `divide()`
`\\`  | `floordiv()`
`%`   | `mod()`
`**`  | `pow()`

## Универсальные функции: выполнение операций <br/> между объектами DataFrame и Series

Операции между DataFrame и Series походят на операции между двухмерным   <br/>
и одномерным массивами, и ты, наверное, понимаешь почему. Разница лишь   <br/>
в том, что транслирование идёт с поправкой на выравнивание. Рассмотрим   <br/>
часто встретяющуюся операцию - вычитание одной строки из матрицы.

In [33]:
A = rng.integers(0, 10, (3, 4))  # а вот и наша матрица
A

array([[3, 2, 4, 9],
       [7, 1, 1, 0],
       [0, 3, 7, 2]])

In [35]:
A - A[0]  # из всех строк вычитаем первую строку

array([[ 0,  0,  0,  0],
       [ 4, -1, -3, -9],
       [-3,  1,  3, -7]])

То же самое происходит и с объектами Pandas.

In [52]:
df = pd.DataFrame(A, columns=list('QRST'))
df - df.iloc[0]  # всё как NumPy завещал
            # можно было подумать, что вычитаться будут только объекты Series с общими метками строк.
            # И это правда, но только не здесь, когда у нас всего одна строка. Здесь происходит
            # полноценное транслирование. Если бы у тебя было несколько строк, то правила транслирования
            # бы не работали, и применялось выравнивание. Моя догадка.

,Q,R,S,T
0,0,0,0,0
1,4,-1,-3,-9
2,-3,1,3,-7


Можно также выполнять операции и по столбцам, не забыв указать `axis`.

In [61]:
df.subtract(df['R'], axis=0)  # по умолчанию вычитание бьёт по столбцам, ведь это же DataFrame.
                       # Но нам переключаем удар на строки, после чего происходит выравнивание
                       # по индексам строк и транслирование по всем стобцам. Это нужно представить.
                       #****всё-равно непонятно. Скорее всего у pd.DataFrame'ов 0-вой осью считаются
                       #    столбы

,Q,R,S,T
0,1,0,2,7
1,6,0,0,-1
2,-3,0,4,-1


Если ты читал комментарий, то понимаешь когда происходит выравнивание.

In [59]:
df

,Q,R,S,T
0,3,2,4,9
1,7,1,1,0
2,0,3,7,2


In [69]:
df.add(df.loc[[0, 2], ['R', 'S']], axis=0)  # axis=0 - здесь это можно было опустить
                                            #****df.loc[[0, 2], ['R', 'S']] скорее всего выдаёт нам
                                            #        ﹇ ﹇
                                            #        2  4
                                            #        3  7
                                            #        ﹈ ﹈

,Q,R,S,T
0,NaN,4.0,8.0,NaN
1,NaN,NaN,NaN,NaN
2,NaN,6.0,14.0,NaN


Такое распределение значений по меткам в Pandas всегда  <br/>
сохраняет контекс данных при проведении операций между  <br/>
неоднородными / неправельными / неодинаковыми данными.